# Motor Control

Stepper motor control for telescope ALT/AZ positioning.

**⚠️ Motors must be connected and controller boards pre-programmed!**

---

## Quick Start

1. Run **"Setup"** - imports dependencies
2. Run **"List Serial Ports"** - find motor controller  
3. Run **"Motor Controller Class"** - defines `MotorController`
4. Run **"Connect to Motors"** - opens serial connection
5. Run **"Home Position"** - return to safe starting position

---

## Motor Configuration

| Axis | Range | Home | Steps/Degree |
|------|-------|------|--------------|
| ALT | 0° (zenith) → ~90° (horizon) | 0 (zenith) | ~19,167 |
| AZ | ±110,000 steps (~±50°) | 0 (center) | ~2,222 |

![ALT-AZ Coordinates](Altaz-coordinates-64fd7c4.webp "ALT-AZ Coordinates")

---
## Setup

Dependencies (`pyserial`, `astropy`) installed via PDM.

In [ ]:
# ============================================
# SETUP
# Run this first after every kernel restart
# Dependencies installed via PDM (pyproject.toml)
# ============================================

import serial
import serial.tools.list_ports
from typing import Optional
from dataclasses import dataclass
from enum import Enum
import time

print("✓ Setup complete")

# ============================================
# LIST SERIAL PORTS
# Find motor controller port
# ============================================

ports = serial.tools.list_ports.comports()

print("Available ports:")
for port in ports:
    print(f"  {port.device}: {port.description}")
    if port.vid:
        print(f"    VID:PID = {port.vid:04X}:{port.pid:04X}")

# Auto-detect motor controller (usually ACM port)
MOTOR_PORT = None
for port in ports:
    if "ACM" in port.device:
        MOTOR_PORT = port.device
        print(f"\n✓ Auto-selected: {MOTOR_PORT}")
        break

if not MOTOR_PORT and ports:
    MOTOR_PORT = ports[0].device
    print(f"\n⚠️ No ACM port found, using: {MOTOR_PORT}")

---
## Motor Controller Class

The `MotorController` class provides a clean interface to the stepper motor controller board.

# ============================================
# MOTOR CONTROLLER CLASS
# ============================================

class Axis(Enum):
    """Motor axis identifiers."""
    ALT = 0  # Altitude (up/down)
    AZ = 1   # Azimuth (left/right)


@dataclass
class AxisConfig:
    """Configuration for a motor axis."""
    axis: Axis
    steps_per_degree: float
    min_steps: int
    max_steps: int
    home_steps: int = 0


# Default axis configurations
ALT_CONFIG = AxisConfig(
    axis=Axis.ALT,
    steps_per_degree=1725000 / 90.0,  # ~19,167 steps/degree
    min_steps=0,       # Zenith (straight up)
    max_steps=140000,  # Near horizon
    home_steps=0
)

AZ_CONFIG = AxisConfig(
    axis=Axis.AZ,
    steps_per_degree=200000 / 90.0,  # ~2,222 steps/degree
    min_steps=-110000,  # Counter-clockwise limit
    max_steps=110000,   # Clockwise limit
    home_steps=0
)


class MotorController:
    """Interface to telescope stepper motor controller.
    
    Serial Protocol:
        A<n>  - Select axis (0=ALT, 1=AZ)
        o<n>  - Go to absolute position (steps)
        O<n>  - Move relative (steps, +/-)
        ?     - Display help
    """
    
    def __init__(
        self, 
        port: str,
        alt_config: AxisConfig = ALT_CONFIG,
        az_config: AxisConfig = AZ_CONFIG,
        timeout: float = 30.0
    ):
        self.port = port
        self.alt_config = alt_config
        self.az_config = az_config
        self.timeout = timeout
        self._serial: Optional[serial.Serial] = None
        self._current_axis: Optional[Axis] = None
        
    def connect(self) -> None:
        """Open serial connection."""
        self._serial = serial.Serial(self.port, timeout=self.timeout)
        time.sleep(0.5)
        print(f"✓ Connected to motor controller on {self.port}")
        
    def disconnect(self) -> None:
        """Close serial connection."""
        if self._serial:
            self._serial.close()
            self._serial = None
            print("✓ Disconnected from motor controller")
    
    @property
    def is_connected(self) -> bool:
        return self._serial is not None and self._serial.is_open
    
    def _send_command(self, cmd: str, wait_response: bool = True) -> str:
        if not self.is_connected:
            raise RuntimeError("Not connected to motor controller")
        
        self._serial.write(cmd.encode())
        
        if wait_response:
            response = self._serial.read_until(expected=b"{'alldone': 1}\r\n")
            return response.decode('utf-8')
        else:
            response = self._serial.read_until(expected=b"}")
            return response.decode('utf-8')
    
    def _select_axis(self, axis: Axis) -> None:
        if self._current_axis != axis:
            self._send_command(f'A{axis.value}', wait_response=False)
            self._current_axis = axis
    
    def get_help(self) -> list[str]:
        """Get help text from controller."""
        if not self.is_connected:
            raise RuntimeError("Not connected to motor controller")
        
        self._serial.write(b'?')
        response = self._serial.read_until(expected=b'?\tdisplay this help screen\r\n')
        return response.decode('utf-8').splitlines()
    
    # --- Absolute Position ---
    
    def goto_steps(self, axis: Axis, steps: int) -> str:
        """Move axis to absolute position in steps."""
        config = self.alt_config if axis == Axis.ALT else self.az_config
        steps = max(config.min_steps, min(config.max_steps, steps))
        self._select_axis(axis)
        return self._send_command(f'o{steps}')
    
    def goto_alt_steps(self, steps: int) -> str:
        return self.goto_steps(Axis.ALT, steps)
    
    def goto_az_steps(self, steps: int) -> str:
        return self.goto_steps(Axis.AZ, steps)
    
    def goto_degrees(self, axis: Axis, degrees: float) -> str:
        """Move axis to absolute position in degrees."""
        config = self.alt_config if axis == Axis.ALT else self.az_config
        steps = int(degrees * config.steps_per_degree)
        return self.goto_steps(axis, steps)
    
    def goto_alt(self, degrees: float) -> str:
        """Move to altitude in degrees (0=zenith, 90=horizon)."""
        return self.goto_degrees(Axis.ALT, degrees)
    
    def goto_az(self, degrees: float) -> str:
        """Move to azimuth in degrees (0=center, +=CW, -=CCW)."""
        return self.goto_degrees(Axis.AZ, degrees)
    
    # --- Relative Movement ---
    
    def move_steps(self, axis: Axis, steps: int) -> str:
        """Move axis by relative amount in steps."""
        self._select_axis(axis)
        return self._send_command(f'O{steps}')
    
    def move_alt_steps(self, steps: int) -> str:
        return self.move_steps(Axis.ALT, steps)
    
    def move_az_steps(self, steps: int) -> str:
        return self.move_steps(Axis.AZ, steps)
    
    def move_degrees(self, axis: Axis, degrees: float) -> str:
        config = self.alt_config if axis == Axis.ALT else self.az_config
        steps = int(degrees * config.steps_per_degree)
        return self.move_steps(axis, steps)
    
    def move_alt(self, degrees: float) -> str:
        """Move altitude by degrees (+down, -up)."""
        return self.move_degrees(Axis.ALT, degrees)
    
    def move_az(self, degrees: float) -> str:
        """Move azimuth by degrees (+CW, -CCW)."""
        return self.move_degrees(Axis.AZ, degrees)
    
    # --- Convenience ---
    
    def home(self) -> None:
        """Return both axes to home position."""
        print("Homing altitude...")
        self.goto_alt_steps(self.alt_config.home_steps)
        print("Homing azimuth...")
        self.goto_az_steps(self.az_config.home_steps)
        print("✓ Home position reached")
    
    def goto_altaz(self, alt_deg: float, az_deg: float) -> None:
        """Move to specified ALT/AZ coordinates."""
        print(f"Moving to ALT={alt_deg:.2f}°, AZ={az_deg:.2f}°")
        self.goto_alt(alt_deg)
        self.goto_az(az_deg)
        print("✓ Position reached")


print("✓ MotorController class defined")

# ============================================
# CONNECT TO MOTORS
# ============================================

# Override port if needed: MOTOR_PORT = '/dev/ttyACM1'

motors = MotorController(MOTOR_PORT)
motors.connect()

In [ ]:
# ============================================
# TEST CONNECTION
# Verify communication with motor controller
# ============================================

help_text = motors.get_help()
print("Motor controller commands:")
for line in help_text:
    print(f"  {line}")

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
---
## Observatory & GoTo Integration

The `Observatory` class integrates motor control with astronomical coordinates for GoTo functionality.

['ttyACM1', 'ttyACM0']


In [ ]:
# ============================================
# OBSERVATORY CLASS
# ============================================

from astropy.coordinates import EarthLocation, SkyCoord, AltAz
from astropy.time import Time
from datetime import datetime, timezone
from typing import NamedTuple


class ObserverLocation(NamedTuple):
    """Observer location on Earth."""
    name: str
    latitude: float   # degrees, positive North
    longitude: float  # degrees, positive West (for US)
    elevation: float  # meters above sea level
    tz_name: str      # IANA timezone string


DEFAULT_LOCATION = ObserverLocation(
    name="Verona, WI",
    latitude=42.9917,
    longitude=-89.5332,
    elevation=320.0,
    tz_name="America/Chicago"
)


class Observatory:
    """Telescope observatory with GoTo capability.
    
    Converts celestial coordinates (RA/DEC) to motor positions.
    
    Example:
        >>> obs = Observatory(motors, DEFAULT_LOCATION)
        >>> obs.goto_radec(SkyCoord('5h15m35s', '-8d10m37s'))  # Rigel
        >>> obs.goto_object("M42")  # Orion Nebula
    """
    
    def __init__(
        self,
        motors: MotorController,
        location: ObserverLocation = DEFAULT_LOCATION,
        az_heading: float = 180.0  # South-facing default
    ):
        self.motors = motors
        self.location = location
        self.az_heading = az_heading
        
        self._earth_location = EarthLocation(
            lat=location.latitude,
            lon=location.longitude,
            height=location.elevation
        )
        
        print(f"✓ Observatory: {location.name}")
        print(f"  {location.latitude:.4f}°N, {location.longitude:.4f}°W")
    
    def _get_altaz(
        self, 
        coord: SkyCoord, 
        obstime: Optional[datetime] = None
    ) -> tuple[float, float]:
        """Convert sky coordinates to ALT/AZ."""
        if obstime is None:
            obstime = datetime.now(timezone.utc)
        
        altaz_frame = AltAz(
            location=self._earth_location,
            obstime=Time(obstime)
        )
        target_altaz = coord.transform_to(altaz_frame)
        
        return float(target_altaz.alt.degree), float(target_altaz.az.degree)
    
    def _altaz_to_motor_steps(
        self, 
        alt_deg: float, 
        az_deg: float
    ) -> tuple[int, int]:
        """Convert ALT/AZ to motor steps."""
        # ALT: 0 steps = zenith (90°), increases toward horizon
        alt_steps = int(
            (90.0 - alt_deg) * self.motors.alt_config.steps_per_degree
        )
        
        # AZ: Convert from compass to motor-relative
        relative_az = self.az_heading - az_deg
        relative_az = (relative_az + 180) % 360 - 180  # Normalize
        az_steps = int(relative_az * self.motors.az_config.steps_per_degree)
        
        return alt_steps, az_steps
    
    def goto_radec(
        self, 
        coord: SkyCoord,
        obstime: Optional[datetime] = None
    ) -> None:
        """Slew telescope to RA/DEC coordinates."""
        alt_deg, az_deg = self._get_altaz(coord, obstime)
        alt_steps, az_steps = self._altaz_to_motor_steps(alt_deg, az_deg)
        
        print(f"Target: RA={coord.ra.to_string(unit='hour', precision=1)}, "
              f"DEC={coord.dec.to_string(precision=1)}")
        print(f"ALT/AZ: {alt_deg:.2f}°, {az_deg:.2f}°")
        print(f"Steps:  ALT={alt_steps}, AZ={az_steps}")
        
        # Check limits
        alt_cfg = self.motors.alt_config
        az_cfg = self.motors.az_config
        
        if not (alt_cfg.min_steps <= alt_steps <= alt_cfg.max_steps):
            print(f"⚠️ ALT outside range [{alt_cfg.min_steps}, {alt_cfg.max_steps}]")
            return
        
        if not (az_cfg.min_steps <= az_steps <= az_cfg.max_steps):
            print(f"⚠️ AZ outside range [{az_cfg.min_steps}, {az_cfg.max_steps}]")
            return
        
        print("Slewing...")
        self.motors.goto_alt_steps(alt_steps)
        self.motors.goto_az_steps(az_steps)
        print("✓ Target acquired")
    
    def goto_object(self, name: str, obstime: Optional[datetime] = None) -> None:
        """Slew to named object (requires internet)."""
        print(f"Looking up '{name}'...")
        try:
            coord = SkyCoord.from_name(name)
            self.goto_radec(coord, obstime)
        except Exception as e:
            print(f"❌ Could not find '{name}': {e}")


print("✓ Observatory class defined")

Serial<id=0x7f876c6260e0, open=True>(port='/dev/ttyACM1', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)

In [ ]:
# ============================================
# CREATE OBSERVATORY
# Initialize with motor controller and location
# ============================================

# Update for your observing site
MY_LOCATION = ObserverLocation(
    name="Verona, WI",
    latitude=42.9917,
    longitude=-89.5332,
    elevation=320.0,
    timezone="America/Chicago"
)

observatory = Observatory(
    motors=motors,
    location=MY_LOCATION,
    az_heading=180.0  # Compass heading when AZ motor is centered
)

---
## Motor Control Demo

Interactive cells to test motor functionality.

In [ ]:
### Home Position

# Return both axes to home (zenith, center)
motors.home()

In [ ]:
### Altitude Motor Tests

# Move ALT to near horizon
print("Moving ALT toward horizon...")
motors.goto_alt_steps(140000)
print("✓ ALT at maximum extension")

In [ ]:
# Return ALT to zenith (home)
print("Returning ALT to zenith...")
motors.goto_alt_steps(0)
print("✓ ALT at home (zenith)")

In [ ]:
# Small ALT nudge down (+)
print("Nudging ALT down...")
motors.move_alt_steps(300)
print("✓ ALT nudged")

In [ ]:
# Small ALT nudge up (-)
print("Nudging ALT up...")
motors.move_alt_steps(-300)
print("✓ ALT nudged")

In [ ]:
### Azimuth Motor Tests

# Rotate AZ counter-clockwise to limit
print("Rotating AZ CCW to limit...")
motors.goto_az_steps(-110000)
print("✓ AZ at CCW limit")

In [ ]:
# Rotate AZ clockwise to limit
print("Rotating AZ CW to limit...")
motors.goto_az_steps(110000)
print("✓ AZ at CW limit")

In [ ]:
# Return AZ to center (home)
print("Returning AZ to center...")
motors.goto_az_steps(0)
print("✓ AZ at home (center)")

In [ ]:
# Small AZ nudge clockwise (+)
print("Nudging AZ CW...")
motors.move_az_steps(700)
print("✓ AZ nudged")

In [ ]:
# Small AZ nudge counter-clockwise (-)
print("Nudging AZ CCW...")
motors.move_az_steps(-700)
print("✓ AZ nudged")

In [ ]:
### GoTo Demo

b"\r\n{'t': 2.00}\r\n{'alldone': 1}\r\n"

In [ ]:
# GoTo using RA/DEC coordinates
# Rigel - bright star in Orion

Rigel = SkyCoord('5h15m35.01s', '-8d10m37.1s')
print("Targeting Rigel...")
observatory.goto_radec(Rigel)

In [ ]:
# GoTo using RA/DEC coordinates
# Betelgeuse - red supergiant in Orion

Betelgeuse = SkyCoord('5h56m20.95s', '7d24m37.8s')
print("Targeting Betelgeuse...")
observatory.goto_radec(Betelgeuse)

b"\r\n{'t': 0.42}\r\n{'alldone': 1}\r\n"

# GoTo by object name (requires internet)
observatory.goto_object("M42")  # Orion Nebula

In [ ]:
# GoTo by object name - other examples

# observatory.goto_object("M31")      # Andromeda Galaxy
# observatory.goto_object("M45")      # Pleiades
# observatory.goto_object("NGC 7000") # North America Nebula
# observatory.goto_object("Polaris")  # North Star

---
## Cleanup

Disconnect from motor controller when done.

In [ ]:
# ============================================
# DISCONNECT
# Close serial connection to motor controller
# ============================================

motors.disconnect()

---
## Reference: Motor Physical Limits

### Altitude (ALT)
- **Range:** 0° (zenith/straight up) to ~90° (horizon)
- **Home:** 0 steps (zenith) - safe storage position
- **Steps:** ~140,000 for full range (~19,167 steps/degree)
- **Limits:** Physical mount limits near horizon; site obstructions (trees, buildings)
- **Physics:** Torque requirements increase as scope tilts due to moment arm

### Azimuth (AZ)
- **Range:** ~270° total rotation (±135° from center)
- **Home:** 0 steps (center) - belt tensioner opposite drive motor
- **Steps:** ~220,000 full range (~2,222 steps/degree)
- **Safe range:** ±110,000 steps (~±50°)
- **Limits:** Belt tension changes at extremes; physical cable routing

### Calibration Notes
- ALT home (zenith) confirmed when OTA is vertical
- AZ center confirmed when belt tensioner opposite motor (feel under scope)
- For GoTo accuracy, align with known star and adjust `az_heading` parameter